In [1]:
import CMR_HFpEF_Analysis.Defaults as Defaults
import CMR_HFpEF_Analysis.functions_collection as ff
import CMR_HFpEF_Analysis.Image_utils as util

import os
import numpy as np
import nibabel as nb
import matplotlib.pyplot as plt
import SimpleITK as sitk

main_path = '/mnt/mount_zc_NAS/HFpEF/data/HFpEF_data/'

## Select ED and ES visually

In [8]:
case_list = ff.find_all_target_files(['ID_0753'], os.path.join(main_path, 'raw_img'))

for case in case_list:

    patient_id = os.path.basename(case)

    # find all frames
    frame_list = ff.sort_timeframe(ff.find_all_target_files(['Org*'],case),1,'e')

    # put the mid slice from each frame into a new image
    for i in range(0,frame_list.shape[0]):
        frame = sitk.ReadImage(frame_list[i])
        spacing = frame.GetSpacing()
        img = np.rollaxis(sitk.GetArrayFromImage(frame),0,3)
        img = np.rollaxis(img,0,2)

        if i == 0:
            m, n, k = img.shape
            mid_slice_image = np.zeros([m,n, frame_list.shape[0]])
        
        img = util.crop_or_pad(img, [m,n,img.shape[-1]])

        # find the non-blank slice
        non_blank_slices = [i for i in range(img.shape[-1]) if np.sum(img[:, :, i]) != 0]

        # put the middle slice into the mid_slice_image
        mid_slice_image[:,:,i] = img[:,:, non_blank_slices[len(non_blank_slices)//2]]
        
    # save this stacked image for visual decision
    save_folder = os.path.join(main_path, 'ED_ES', patient_id); ff.make_folder([save_folder])
    nb.save(nb.Nifti1Image(np.flip(np.flip(mid_slice_image,0),1), np.eye(4)), os.path.join(save_folder, 'mid_slice_img.nii.gz'))

    # prepare a blank txt:
    with open(os.path.join(save_folder, 'ED_ES.txt'), 'w') as file:
        pass  # Do nothing


In [18]:
# after visual selection of ED and ES, how to read:
case_list = ff.find_all_target_files(['ID_0331'], os.path.join(main_path, 'img'))

for case in case_list:

    patient_id = os.path.basename(case)

    filename = os.path.join(main_path, 'ED_ES', patient_id,'ED_ES.txt')

    # prepare a blank txt:
    with open(filename, 'r') as file:
        line = file.readline().strip()
        numbers = line.split()
        num1 = int(numbers[0])
        num2 = int(numbers[1])
        print("Number 1:", num1)
        print("Number 2:", num2)

Number 1: 1
Number 2: 11
